# ID2214/FID3214 Assignment 3 Group no. [1]
### Project members: 
[Patrik Zhong, pzhong@kth.se]
[Edvin Walleborn, edvinw@kth.se]
[Alexander Carlsson, alecarls@kth.se]


### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [1]:
import numpy as np
import pandas as pd
import time
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [2]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"sklearn version: {sklearn.__version__}")

Python version: 3.9.13
NumPy version: 1.23.4
Pandas version: 1.2.5
sklearn version: 1.1.2


## Reused functions from Assignment 1

In [3]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# =========================================== COLUMN FILTER ===================================== #
# =========================================== COLUMN FILTER ===================================== #

globalTestList = [] # only used for print somewhere
def create_column_filter(df):
    dfnew = df.copy()    
    column_filter = []
    for col in dfnew.columns:
        if col == 'CLASS' or col == 'ID':
            column_filter.append(col)
            continue
        
        if dfnew[col].isna().all():
            dfnew.drop(labels=col, axis=1, inplace = True)
        
        elif(dfnew[col].dropna().nunique() <= 1):
            dfnew.drop(col, axis = 1, inplace = True)
        
        else:
            column_filter.append(col)
          
    return dfnew, column_filter

def apply_column_filter(df, column_filter):
    dfnew = df.copy()
    dfnew= dfnew.filter(items = column_filter, axis = 1)

    return dfnew

# =========================================== IMPUTATION ===================================== #
# =========================================== IMPUTATION ===================================== #

def create_imputation(df):
    dfnew = df.copy()
    mappingDict = {}
    #display(dfnew)
    for col in dfnew.columns:
        is_float = pd.api.types.is_float_dtype(dfnew[col].dtypes)
        is_int64 = pd.api.types.is_int64_dtype(dfnew[col].dtypes)
        is_object = pd.api.types.is_object_dtype(dfnew[col].dtypes)
        is_category = pd.api.types.is_categorical_dtype(dfnew[col].dtypes)

        #print(col)
        #print(dfnew[col].dtypes)

        if col == 'CLASS' or col == 'ID':
            continue
        elif (is_int64):
            mean = dfnew[col].mean()
            if pd.isna(mean):
                mean = 0
            dfnew[col].fillna(value=mean, inplace=True)
            mappingDict[col] = (mean)
        elif(is_float):
            mean = dfnew[col].mean()
            if pd.isna(mean):
                mean = 0.0
            dfnew[col].fillna(value=mean, inplace=True)
            mappingDict[col] = (mean)
        elif (is_object):
            mode = dfnew[col].mode(dropna=True)[0]
            if pd.isna(mode):
                mode = ""
            dfnew[col].fillna(value=mode, inplace=True)
            mappingDict[col] = (mode)
        elif(is_category):
            mode = dfnew[col].mode(dropna=True)[0]
            if pd.isna(mode):
                mode = dfnew[col].categories[0]
            dfnew[col].fillna(value=mode, inplace=True)
            mappingDict[col] = (mode)
        else:
            print("no match on dtype in if-statement in (create_imputation)")

    #display(dfnew)
    return dfnew, mappingDict


def apply_imputation(df, imputation):
        dfnew = df.copy()
        for col in dfnew.columns:
            if col == 'CLASS' or col == 'ID':
                continue
            else:
                dfnew[col].fillna(value=imputation[col], inplace=True)
        return dfnew

# =========================================== ONE-HOT ===================================== #
# =========================================== ONE-HOT ===================================== #

def create_one_hot(df):
    dfnew = df.copy()
    catDict = {}
    # display(dfnew)
    
    for col in dfnew.columns:
        not_object = not pd.api.types.is_object_dtype(dfnew[col].dtypes)
        not_category = not pd.api.types.is_categorical_dtype(dfnew[col].dtypes)
        if col == 'CLASS' or col == 'ID' or not_object and not_category:
            continue
        #turns out that almost all columns are objects, not categories. Cast objects to categories with astype.
        # categories = pd.Categorical(dfnew[col])
        dfnew[col] = dfnew[col].astype("category")
        #varje col får tre nya kolumner för varje kategori (b, o, x)
        dummyVars = pd.get_dummies(dfnew[col])
        #appenda col + [b, o, x] som en egen kolumn
        catList = []
        for catcol in dummyVars:
            catList.append(catcol)
            newName = col + "-" + catcol
            dfnew[newName] = dummyVars[catcol]
            dfnew[newName] = dfnew[newName].astype('float')
        #drop the column after worked through the categories
        dfnew = dfnew.drop(col, axis=1)
    #convert everything to float
        catDict[col] = catList
           
    # display(dfnew)
    # print(catDict)        
    
    return dfnew, catDict

def apply_one_hot(df, one_hot):
    dfnew = df.copy()

    # for col in dfnew.columns

    #     dfnew[col] = dfnew[col].astype("category")

    for col in one_hot: #för varje key i vår dict
        not_object = not pd.api.types.is_object_dtype(dfnew[col].dtypes)
        not_category = not pd.api.types.is_categorical_dtype(dfnew[col].dtypes)
        if col == 'CLASS' or col == 'ID' or not_object and not_category:
            continue
        dummyVars = pd.get_dummies(dfnew[col]) #skapa dummyvariablerna
        for catcol in one_hot[col]: #för varje kategori, appenda kategorin med dummyvariabeln
            newName = col + "-" + catcol
            dfnew[newName] = dummyVars[catcol].astype('float')
        dfnew = dfnew.drop(col, axis=1)


    return dfnew

def accuracy(df, correctlabels):
    dfnew = df.copy()
    maxIndex = dfnew.idxmax(axis=1)
    #print(maxIndex)
    #print(len(dfnew.columns))
    
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for i, elem in enumerate(maxIndex):
        if maxIndex[i] == correctlabels[i]:
            tp += 1
        else:
            fp += 1
        
    accuracy = (tp+tn)/(tp+fp+tn+fn)#(len(dfnew.columns) * len(dfnew.index))
    #print("counter: {}".format(counter))
    #print("tp: {}, tn: {}, fp: {}, fn: {}".format(tp, tn, fp, fn))

    #print(maxIndex)
    return accuracy

# =========================================== FOLDS ===================================== #
# =========================================== FOLDS ===================================== #

def folds(df, nofolds):
    dfnew = df.copy()
    output = []
    dflist = np.random.permutation(df.index)
    
    splits = np.array_split(dflist, nofolds, axis=0)
    output = []
    for subArray in splits:
        # print(subArray)
        tempdf = pd.DataFrame(subArray)
        output.append(tempdf)
    return output

# =========================================== BRIER ===================================== #
# =========================================== BRIER ===================================== #

def brier_score(df, correctlabels):
    dfnew = df.copy()

    #print(correctlabels)
    #display(df)
    
    counter = 0
    score = 0
    for i, row in enumerate(dfnew.iterrows()):
     
        correctIndex = np.where(df.columns==correctlabels[i])[0][0]
        #print("correctIndex: {}".format(correctIndex))

        for j, col in enumerate(row[1]):
            counter += 1
            #print("col: {}".format(col))


            if j == correctIndex:
                score += (col-1)**2
            else:
                score += (col)**2

    
    # score /= counter

    return score/df.index.size

## 1. Define the class RandomForest

In [4]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#a
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) 
#  to None: column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a 
#                      bootstrap sample and the number of evaluated features is log2|F| where |F| is the 
#                      total number of features (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping
#         the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class 
#               probabilities are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, 
#         imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method 
#         predict_proba(X) where X are the (numerical) values of the transformed dataframe; 
#         you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each 
#         class label, to which you add the prediction of each tree on each iteration, and then finally 
#         divide the prediction matrix by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all 
#         possible class labels and hence the prediction of each tree will contain probabilities for all 
#         class labels (in the same order). Note that this assumption may be violated, and this limitation 
#         will be addressed in the next part of the assignment. 

class RandomForest:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees = 100):
        dfnew, self.column_filter = create_column_filter(df)
        dfnew, self.imputation = create_imputation(dfnew)
        dfnew, self.one_hot = create_one_hot(dfnew)
        self.labels = df['CLASS'].astype("category")
        
        dfnew.drop(columns = ["CLASS"], inplace = True)
        numericals = dfnew.to_numpy()        
        rng = np.random.default_rng()

        self.model = []

        for i in range(no_trees):
            bootstrap = rng.choice(numericals.shape[0], len(numericals),replace=True)
            X_bootstrap = numericals[bootstrap]
            X_bootstrap = numericals[bootstrap]
            y_bootstrap = self.labels[bootstrap]          


            
            dt = DecisionTreeClassifier(max_features = "log2")
            dt.fit(X_bootstrap, y_bootstrap)
            self.model.append(dt)           

    def predict(self, df):

        #hint 1
        dfnew = df.drop(columns = ['CLASS'])
        dfnew = apply_column_filter(dfnew, self.column_filter)
        dfnew = apply_imputation(dfnew, self.imputation)
        dfnew = apply_one_hot(dfnew, self.one_hot)

        #hint 2
        matrix = np.zeros((len(dfnew), len(pd.unique(self.labels))))
             
        for tree in self.model:
            pred = tree.predict_proba(dfnew.to_numpy())
            matrix+=pred #add the pred (matrix) to matrix, accumulates from previous trees

        matrixDiv = matrix/len(self.model)
        predictionsdf = pd.DataFrame(matrixDiv, columns = pd.unique(pd.unique(self.labels)))
        print(predictionsdf)
            
        return predictionsdf


In [5]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
# print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.26 s.
     negative  positive
0        0.39      0.61
1        0.19      0.81
2        0.14      0.86
3        0.63      0.37
4        0.17      0.83
..        ...       ...
474      0.17      0.83
475      0.41      0.59
476      0.18      0.82
477      0.24      0.76
478      0.58      0.42

[479 rows x 2 columns]
Testing time: 0.05 s.
Accuracy: 0.9040
Brier score: 0.1857


In [6]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
# print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

     negative  positive
0        0.80      0.20
1        0.81      0.19
2        0.93      0.07
3        0.04      0.96
4        0.05      0.95
..        ...       ...
474      0.03      0.97
475      0.79      0.21
476      0.15      0.85
477      0.12      0.88
478      0.88      0.12

[479 rows x 2 columns]
Accuracy on training set: 1.0000
Brier score on training set: 0.0220


### Comment on assumptions, things that do not work properly, etc.



## 2a. Handling trees with non-aligned predictions

In [7]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 



In [8]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.39 s.


KeyError: 'C'

## 2b. Estimate predictive performance using out-of-bag predictions

In [8]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively. By "aggregated out-of-bag predictions" is here meant the sum of all 
#         predicted probabilities (one sum per class and instance). These sums should be divided by the number of predictions
#         (stored in the vector) in order to obtain a single class probability distribution per training instance. 
#         This distribution is considered to be the out-of-bag prediction for each instance, and e.g., the class that 
#         receives the highest probability for each instance can be compared to the correct label of the instance, 
#         when calculating the accuracy using the out-of-bag predictions.
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector
#
#         For example, assuming that we have two class labels, then we may end up with the following matrix:
#
#         2 4
#         4 4
#         5 0
#         ...
#
#         and the vector (no. of predictions) (6, 8, 5, ...)
#
#         The resulting class probability distributions are:
#
#         0.333... 0.666...
#         0.5 0.5
#         1.0 0

In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 1.96 s.
OOB accuracy: 0.9555
Testing time: 0.06 s.
Accuracy: 0.9488
AUC: 0.9718
Brier score: 0.0986


In [10]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d5b21b1a-ec26-41e0-8ae3-fe1bc5b86ef4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>